
Created on Fri Dec 11 19:01:34 2020

@author: Michal Oruba

# Wstęp

Analizowanym zbiorem jest Titanic, który został zapisany w pliku o rozszerzeniu *.tsc*. Jest to rozszerzenie, które oddziela kolejne kolumny przy pomocy tabulatora. Dostarczony plik zawierał informacje, które nie będą wymagane podczas analizy, takie które były błędne, lub ich brak. W dalszej części dokumentu, przedstawione zostaną wszystkie nieprawidłowości, które zostały odnalezione w zbiorze danych, a także sposoby i rezultaty ich poprawy.

Kolumny zawarte w zbiorze danych oraz ich opis zostały przedstawione poniżej:

| Nazwa         | Opis                                         |
| --------------|----------------------------------------------|
| PassengerId   | Unikalny identyfikator pasażera              |
| Survived      | Przeżył (1), zmarł (0)                       |
| Pclass        | Klasa pasażera                               |
| Name          | Imię pasażera                                |
| Sex           | Płeć pasażera                                |
| Age           | Wiek pasażera                                |
| SibSp         | Liczba rodzeństwa lub małżonków na pokładzie |
| Parch         | liczba rodziców lub dzieci na pokładzie      |
| Ticket        | Numer biletu                                 |
| Fare          | Cena biletu                                  |
| Cabin         | Numer kabiny                                 |
| Embarked      | Port wypłynięcia pasażera                    |
| Ship          | Nazwa statku			                       |

# Analiza zbioru danych

Do przeglądania oraz przetwarzania zbioru danych wykorzystano bibliotekę *Pandas*. Aby wczytać plik z danymi należało skorzystać z polecenia *pd.read_csv()*. Następnie, czy pomocy polecenia *columns.tolist()* uzyskano informację na temat kolumn, które znajdują się w pliku, co potwierdza informacje przedstawione w tabeli przedstawionej we wstępie. 

Aby poprawnie wczytać plik z danymi do tego notatnika, musi on być w tym samym katalogu co ten Notebook, lub należy zmienić ścieżkę na własną.

In [161]:
import pandas as pd

tit = pd.read_csv(r'TitanicMess.tsv', sep='\t', index_col='PassengerId')

In [162]:
print(tit.columns.tolist())

['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked', 'ship']


Aby zweryfikować, jakie typy danych znajdują się w każdej z kolumn, użyto polecenia dtypes. Jego wyniki znajdują się poniżej. Na ich podstawie można wnioskować, że kolumny Age i Fare posiadają niepoprawne lub puste wartości, ponieważ automatycznie został dobrany typ *object*.

In [163]:
print(tit.dtypes)

Survived     int64
Pclass       int64
Name        object
Sex         object
Age         object
SibSp        int64
Parch        int64
Ticket      object
Fare        object
Cabin       object
Embarked    object
ship        object
dtype: object


Aby zweryfikować, które kolumny zawierają puste wartości i jaka jest ich liczba, skorzystano z metody *isnull().sum()*. W wynikach można zaobserwować braki w trzech kolumnach. Są to *Age* (173), *Cabin* (685) oraz *Embarked* (2).

In [164]:
print(tit.isnull().sum())

Survived      0
Pclass        0
Name          0
Sex           0
Age         173
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       685
Embarked      2
ship          0
dtype: int64


Kolejną wykonaną analizą danych była weryfikacja przy pomocy polecenia *describe()*. Wynikiem jest zbiór danych z wyszczególnionymi parametrami takimi jak liczba, liczba unikatowych wartości, element najczęściej występujący oraz liczba jego wystąpień.

In [165]:
print(tit.describe(include=['object']))

                                      Name   Sex  Age Ticket  Fare Cabin  \
count                                  892   892  719    892   892   207   
unique                                 889     6   93    680   250   145   
top     Sandstrom, Miss. Marguerite Ru&5$$  male   24   1601  8,05    G6   
freq                                     3   576   30      7    43     6   

       Embarked     ship  
count       890      892  
unique        6        1  
top           S  Titanic  
freq        643      892  


Dzięki tej informacji można podejrzewać bazując na kolumnie *Name* i wartości *Sandstrom, Miss. Marguerite Ru&5$$*, że zbiór posiada przynajmniej 3 zduplikowane wiersze, ponieważ wartość ta występuje trzykrotnie. Błędem w danych mogą być także znaki specjalne, znajdujące się w najczęściej występującej wartości kolumny *Name*. Dodatkowo informacja o sześciu unikatowych wartościach w kolumnie *Sex* może także wskazywać na błędne wartości, najprawdopodobniej literówki. Także w kolumnie *Embarked* można zauważyć 6 unikatowych wartości, a powinno ich być tylko 3.

Ostatnią informacją, którą można zaobserwować dla wyników polecenia describe jest to, że w kolumnie *Ship* występuje jedna unikatowa wartość. Oznacza to, że nie wnosi ona żadnych informacji do analizy.

Bardziej szczegółowych oględzin będą wymagały kolumny Age oraz Fare. Są to kolumny, które powinny zawierać dane liczbowe, jednak dtypes określa je jako *Object*, co wskazuje na błędne wartości. Należy uzupełnić brakujące dane oraz zweryfikować, czy występują jakieś niepoprawne wartości: błędny format liczb, zły separator dziesiętny, wartości spoza oczekiwanego zakresu.

Puste wartości w kolumnie Cabin nie będą uzupełniane, ze względu na ich liczebność. Uzupełnienie tak dużej liczby braków moze negatywnie wpłynąć na analizę. Nie uzupełniając danych, podczas analizy będzie można zwrócić uwagę na to, czy brak danych można generować jakąś zależność w tym przypadku.

# Czyszczenie danych

Czyszenie danych polegać będzie na usunięciu niepoprawnych lub niepotrzebnych danych, uzupełnieniu brakujących tam, gdzie to możliwe. Podjeta zostanie także próba poprawienia tych informacji, których będzie możliwość. Zmieniony zostanie typ kolumny Age oraz Fare oraz duplikaty będą usunięte.

## Usunięcie kolumny ship

Pierwszym elementem, z którego zostanie oczyszczony zbiór danych, to kolumna *ship*. Jest ona nieprzydatna do dalszej analizy, ze względu na jednakową wartość w każdym wierszu. Nową wartość zapisana została do nowej zmiennej *tit_wo_ship*.

In [166]:
tit_wo_ship = tit.drop(columns='ship')
print(tit_wo_ship.columns.tolist())

['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


## Poprawa kolumny Age

W celu poprawy kolumny *Age* skorzystano z funkcji unique(), do zweryfikowania unikatowych wartości.

In [167]:
print(tit_wo_ship['Age'].unique())

['22' '38' '26' '35' nan '54' '2' '27' '14' '4' '58' '20' '55' '31' '34'
 '15' '8' '19' '40' '.9' '66' '28' '42' '21' '18' '3' '7' '49' '29' '65'
 '28,5' '5' '11' '45' '17' '32' '16' '25' '0,83' '30' '33' '23' '24' '46'
 '59' '71' '37' '47' '14,5' '70,5' '32,5' '12' '9' '36,5' '51' '55,5'
 '40,5' '44' '1' '61' '56' '50' '36' '45,5' '4435' '20,5' '62' '41' '52'
 '63' '23,5' '0,92' '43' '60' '39' '10' '64' '13' '48' '0,75' '-3' '-12'
 '53' '57' '80' '250' '70' '24,5' '6' '0,67' '30,5' '0,42' '34,5' '74']


Aby doprowadzić do możliwości zamiany typu w tej kolumnie na *float*, należy uzupełnić puste wartości, oraz poprawić separator dziesiętny.

Praca zostanie wykonana na kopii bieżącego zbioru danych do zmiennej *tit_correct_age*.

In [168]:
tit_correct_age = tit_wo_ship.copy()

Następnie odnaleziona zostanie mediana za pomocą funkcji *mode()*, którą zastąpione będą puste wartości.

In [169]:
mediana = tit_correct_age['Age'].mode()
print('Mediana wynosi: ' + str(mediana[0]))
tit_correct_age['Age'] = tit_correct_age['Age'].fillna(str(mediana[0]))
print('Liczba pustych wartości dla kolumny Age:')
print(tit_correct_age['Age'].isnull().sum())

Mediana wynosi: 24
Liczba pustych wartości dla kolumny Age:
0


W kolejnym kroku separator dziesiętny w formie przecinka zostanie zamieniony na kropkę.

In [170]:
tit_correct_age['Age'] = tit_correct_age['Age'].str.replace(",",".")

Następnie zmieniony zostanie typ danych w kolumnie na float.

In [171]:
tit_correct_age['Age'] = tit_correct_age['Age'].astype(float)

Do wyznaczenia maksymalnej i minimalnej wartości zostaną wykorzystane metody *max()* i *min()*.

In [172]:
print("Wartość maksymalna: " + str(tit_correct_age['Age'].max()))
print("Wartość minimalna: " + str(tit_correct_age['Age'].min()))

Wartość maksymalna: 4435.0
Wartość minimalna: -12.0


Wyznaczony wiek maksymalny i minimalny posiada błędne wartości. Poniżej odnalezione zostały wszystkie wartości w kolumnie *Age*, których wartości sa ujemne lub większe niż 120.

In [173]:
print(tit_correct_age[tit_correct_age['Age']<1][['Age', 'SibSp', 'Parch']])
print(tit_correct_age[tit_correct_age['Age']>120][['Age', 'SibSp', 'Parch']])

               Age  SibSp  Parch
PassengerId                     
32            0.90      1      0
79            0.83      0      2
306           0.92      1      2
470           0.75      2      1
508          -3.00      0      0
523         -12.00      0      0
645           0.75      2      1
756           0.67      1      1
804           0.42      0      1
832           0.83      1      1
                Age  SibSp  Parch
PassengerId                      
224          4435.0      0      0
667           250.0      0      0


W zbiorze danych istnieją dwie wartości, które posiadają wiek większy niż 100, dwie wartości ujemne, oraz 8 dodatnich, ale mniejszych niż 1.

Wartości ujemne zostaną zamienione na miedianę. Ze względu na brak rodziców (kolumna *Parch*) na pokładzie nie można wnioskować, że znak "-" został dodany przypadkowo. Dzieci w wieku 3 i 12 lat nie powinny podróżować same.

Wartości większe niż 120 również zostaną zamienione na medianę, nie można na ich podstawie wywnioskować, jaka wartość byłaby poprawna.

Wartości z przedziału (0, 1) zostaną zamienione na 1.

In [174]:
# Zmiana wieku powyżej 120 na medianę
tit_correct_age.loc[tit_correct_age['Age'] > 120, 'Age'] = float(mediana)
# Zmiana wieku poniżej 0 na medianę
tit_correct_age.loc[tit_correct_age['Age'] < 0, 'Age'] = float(mediana)
# Zmiana wieku ponizej 1 na 1
tit_correct_age.loc[tit_correct_age['Age'] < 1, 'Age'] = float(1)

print('W zbiorze nie występuje już wiek mniejszy od 1 lub większy od 120:')
print(tit_correct_age[tit_correct_age['Age']<1][['Age', 'SibSp', 'Parch']])
print(tit_correct_age[tit_correct_age['Age']>120][['Age', 'SibSp', 'Parch']])

W zbiorze nie występuje już wiek mniejszy od 1 lub większy od 120:
Empty DataFrame
Columns: [Age, SibSp, Parch]
Index: []
Empty DataFrame
Columns: [Age, SibSp, Parch]
Index: []


## Poprawa kolumny Sex

Podobnie jak w przypadku kolumny *Age* skorzystano z metody *unique()*, aby znaleźć błędne typy płci w kolumnie *Sex*:

In [175]:
print(tit_wo_ship['Sex'].unique())

['male' 'female' 'malef' 'mal' 'fem' 'femmale']


Można zaobserwować, że dla płci męskiej, poza poprawną wartością *male* pojawiają się także *malef* i *mal*. Natomiast dla wartości *female*: *fem* oraz *femmale*. Wartości te zostaną poprawione na właściwe. W pierwszej kolejności zostanie jednak utworzona nowa zmienna *tit_correct_sex*, będąca kopią poprawionego zbioru danych *tit_correct_age*.

In [176]:
tit_correct_sex = tit_correct_age.copy();

tit_correct_sex['Sex'] = tit_correct_sex['Sex'].str.replace(r"\bmalef\b", "male")
tit_correct_sex['Sex'] = tit_correct_sex['Sex'].str.replace(r"\bmal\b", "male")
tit_correct_sex['Sex'] = tit_correct_sex['Sex'].str.replace(r"\bfemmale\b", "female")
tit_correct_sex['Sex'] = tit_correct_sex['Sex'].str.replace(r"\bfem\b", "female")

print('Poprawione wartości kolumny Sex:')
print(tit_correct_sex['Sex'].unique())

Poprawione wartości kolumny Sex:
['male' 'female']


## Poprawa kolumny Embarked

Również w tym przypadku skorzystano z metody *unique()*:

In [177]:
print(tit_correct_sex['Embarked'].unique())

['S' 'C' 'Q' 'So' nan 'Co' 'Qe']


Poprawne wartości dla tej kolumny, to "S", "C", "Q". Pozostałe należy zamienić na te wynienione lub uzupełnić puste wartości. Zdecydowano, że brak wartości zostanie uzupełniony poprzez wartość najczęściej występującą. Do wskazania tej wartości wykorzystano metodę *mode()*.

Aby przeprowadzić poprawę utworzono kopię danych w nowej zmiennej tit_correct_embarked.

In [178]:
tit_correct_embarked = tit_correct_sex.copy();

Następnie wykonano poprawę danych:

In [179]:
tit_correct_embarked['Embarked'] = tit_correct_embarked['Embarked'].str.replace(r"\bSo\b", "S")
tit_correct_embarked['Embarked'] = tit_correct_embarked['Embarked'].str.replace(r"\bCo\b", "C")
tit_correct_embarked['Embarked'] = tit_correct_embarked['Embarked'].str.replace(r"\bQe\b", "Q")

most_common_port = tit_correct_embarked['Embarked'].mode()
print('Najczęściej występujący port to: ' + most_common_port[0])
tit_correct_embarked['Embarked'] = tit_correct_embarked['Embarked'].fillna(most_common_port[0])

print('Poprawione wartości kolumny Embarked:')
print(tit_correct_embarked['Embarked'].unique())

Najczęściej występujący port to: S
Poprawione wartości kolumny Embarked:
['S' 'C' 'Q']


## Usunięcie duplikatów

Aby zweryfikować, czy w danych istnieją duplikaty, skorzystano z metody duplicated(). Wykorzystano w niej parametr *keep=False*, aby w wyniku zwrócić tylko powtarzające się wartości.

Zduplikowane wartości odnaleziono na podstawie wartości indeksu kolumny *PassengerId*.

In [180]:
print(tit_correct_embarked[tit_correct_embarked.index.duplicated(keep=False)])

             Survived  Pclass                                Name     Sex  \
PassengerId                                                                 
11                  1       3  Sandstrom, Miss. Marguerite Ru&5$$  female   
11                  1       3  Sandstrom, Miss. Marguerite Ru&5$$  female   
11                  1       3  Sandstrom, Miss. Marguerite Ru&5$$  female   
225                 1       1        Hoyt, Mr. Frederick Maxfield    male   
225                 1       1        Hoyt, Mr. Frederick Maxfield    male   
225                 1       1       Hoytt, Mr. Frederick Maxfield    male   

              Age  SibSp  Parch   Ticket  Fare Cabin Embarked  
PassengerId                                                    
11            4.0      1      1  PP 9549  16,7    G6        S  
11            4.0      1      1  PP 9549  16,7    G6        S  
11            4.0      1      1  PP 9549  16,7    G6        S  
225          38.0      1      0    19943    90   C93        S  

Zduplikowane były rekordy o ID 11 oraz 225. Jeden z wpisów o ID 225 posiadał błędną wartość w nazwisku, gdzie pojawiła się dwukrotnie litera "t". Aby móc usunąć wszystkie duplikaty jedną metodą, należy najpierw poprawić błędne nazwisko. Zmiany w tym podrozdziale będą realizowane na nowej zmiennej o nazwie *tit_removed_duplicates*.

In [181]:
tit_removed_duplicates = tit_correct_embarked.copy()
tit_removed_duplicates.loc[tit_removed_duplicates.Name == "Hoytt, Mr. Frederick Maxfield", 'Name'] = "Hoyt, Mr. Frederick Maxfield"

print(tit_removed_duplicates[tit_removed_duplicates.index == 225])

             Survived  Pclass                          Name   Sex   Age  \
PassengerId                                                               
225                 1       1  Hoyt, Mr. Frederick Maxfield  male  38.0   
225                 1       1  Hoyt, Mr. Frederick Maxfield  male  38.0   
225                 1       1  Hoyt, Mr. Frederick Maxfield  male  38.0   

             SibSp  Parch Ticket Fare Cabin Embarked  
PassengerId                                           
225              1      0  19943   90   C93        S  
225              1      0  19943   90   C93        S  
225              1      0  19943   90   C93        S  


Aby usunąć duplikaty skorzystano z metody *drop_duplicates*.

In [182]:
tit_removed_duplicates = tit_removed_duplicates.drop_duplicates(inplace=False)

print(tit_removed_duplicates['Name'].describe(include=['object']))

count                                      888
unique                                     888
top       Ford, Miss. Doolina Margaret "Daisy"
freq                                         1
Name: Name, dtype: object


Zauważyć można, że cztery wartości zostały usunięte, było 892, jest 888.

## Poprawa znaków specjalnych w kolumnie Name

Podczas usuwania duplikatów, zaobserwować można było, że pasażer o ID 11 posiada w kolumnie *Name* znaki specjalne. Aby je odszukać wykorzystano metodę *contains()* oraz wyrażenia regularego. Tak jak poprzednio, utworzono też nową kopię danych w zmiennej tit_correct_name.

In [183]:
tit_correct_name = tit_removed_duplicates.copy()
tit_bad_names = tit_correct_name[tit_correct_name['Name'].str.contains('[@_!#$%^&*<>?/\|}{~:]')]

print(tit_bad_names)

             Survived  Pclass                                Name     Sex  \
PassengerId                                                                 
11                  1       3  Sandstrom, Miss. Marguerite Ru&5$$  female   
407                 0       3    Widegren, Mr. Carl/Charles Peter    male   

              Age  SibSp  Parch   Ticket  Fare Cabin Embarked  
PassengerId                                                    
11            4.0      1      1  PP 9549  16,7    G6        S  
407          51.0      0      0   347064  7,75   NaN        S  


W zbiorze danych występują tylko dwie wartości, które posiadają znaki specjalne. Pierwszy to znaleziony wcześniej pasażer o ID 11. A drugi posiada ID 407. Zawiera on jednak tylko znak "/", który jest tutaj użyty prawidłowo. Zmianie będzie podlegać tylko pasażer o ID 11.

Zweryfikowano, że poprawne drugie imie pasażera to "Rut" i na takie zostanie zmieniona wartość.

In [184]:
tit_correct_name.loc[tit_correct_name.Name == "Sandstrom, Miss. Marguerite Ru&5$$", 'Name'] = "Sandstrom, Miss. Marguerite Rut"

print(tit_correct_name[tit_correct_name.index == 11])

             Survived  Pclass                             Name     Sex  Age  \
PassengerId                                                                   
11                  1       3  Sandstrom, Miss. Marguerite Rut  female  4.0   

             SibSp  Parch   Ticket  Fare Cabin Embarked  
PassengerId                                              
11               1      1  PP 9549  16,7    G6        S  


## Poprawa kolumny Fare

W celu poprawy kolumny *Fare* skorzystano z funkcji *unique()*, do zweryfikowania unikatowych wartości.

In [185]:
print(tit_correct_name['Fare'].unique())

['7,25' '71,2833' '7,925' '53,1' '8,05' '8,4583' '51,8625' '21,075'
 '11,1333' '30,0708' '16,7' '26,55' '7,8542' '16' '29,125' '13' '18'
 '7,225' '26' '8,0292' '31,3875' '263' '7,8792' '7,8958' '27,7208'
 '146,5208' '7,75' '10,5' '82,1708' '52' '7,2292' '11,2417' '9,475' '21'
 '41,5792' '15,5' '21,6792' '17,8' '39,6875' '7,8' '76,7292' '61,9792'
 '35,5' '27,75' '46,9' '80' '83,475' '27,9' '15,2458' '8,1583' '8,6625'
 '73,5' '14,4542' '56,4958' '7,65' '29' '12,475' '9' '9,5' '7,7875' '47,1'
 '15,85' '34,375' '61,175' '20,575' '34,6542' '63,3583' '23' '77,2875'
 '8,6542' '7,775' '24,15' '9,825' '14,4583' '247,5208' '31,275' '7,1417'
 '22,3583' '6,975' '7,05' '14,5' '15,0458' '26,2833' '9,2167' '79,2'
 '6,75' '11,5' '36,75' '7,7958' '12,525' '66,6' '7,3125' '61,3792'
 '7,7333' '69,55' '16,1' '15,75' '20,525' '55' '25,925' '33,5' '30,6958'
 '25,4667' '28,7125' '0' '15,05' '39' '22,025' '50' '8,4042' '6,4958'
 '10,4625' '18,7875' '31' '113,275' '27' '76,2917' '90' '9,35' '13,5'
 '7,55' '26,

Dostrzec można, że wartości posiadają różną liczbę miejsc po przecinku. Dodatkowo wszystkie posiadają przecinki jako separatory dziesiętne. Należy zweryfikować, czy występują znaki nieliczbowe, a także czy w zbiorze najdują się liczby ujemne.

W pierwszej kolejności skopiowano zbiór do nowej zmiennej *tit_correct_fare*, a następnie poprawiono przecinki na kropki.

In [186]:
tit_correct_fare = tit_correct_name.copy();
tit_correct_fare['Fare'] = tit_correct_fare['Fare'].str.replace(",",".")

print(tit_correct_fare['Fare'].unique())

['7.25' '71.2833' '7.925' '53.1' '8.05' '8.4583' '51.8625' '21.075'
 '11.1333' '30.0708' '16.7' '26.55' '7.8542' '16' '29.125' '13' '18'
 '7.225' '26' '8.0292' '31.3875' '263' '7.8792' '7.8958' '27.7208'
 '146.5208' '7.75' '10.5' '82.1708' '52' '7.2292' '11.2417' '9.475' '21'
 '41.5792' '15.5' '21.6792' '17.8' '39.6875' '7.8' '76.7292' '61.9792'
 '35.5' '27.75' '46.9' '80' '83.475' '27.9' '15.2458' '8.1583' '8.6625'
 '73.5' '14.4542' '56.4958' '7.65' '29' '12.475' '9' '9.5' '7.7875' '47.1'
 '15.85' '34.375' '61.175' '20.575' '34.6542' '63.3583' '23' '77.2875'
 '8.6542' '7.775' '24.15' '9.825' '14.4583' '247.5208' '31.275' '7.1417'
 '22.3583' '6.975' '7.05' '14.5' '15.0458' '26.2833' '9.2167' '79.2'
 '6.75' '11.5' '36.75' '7.7958' '12.525' '66.6' '7.3125' '61.3792'
 '7.7333' '69.55' '16.1' '15.75' '20.525' '55' '25.925' '33.5' '30.6958'
 '25.4667' '28.7125' '0' '15.05' '39' '22.025' '50' '8.4042' '6.4958'
 '10.4625' '18.7875' '31' '113.275' '27' '76.2917' '90' '9.35' '13.5'
 '7.55' '26.

Kolejnym krokiem jest znalezienie wartości, które nie są liczbowe, np. litery, znaki specjalne.

In [187]:
tit_bad_fare = tit_correct_fare['Fare'][tit_correct_fare['Fare'].str.contains('[a-zA-Z@_!#$%^&*()<>?/|}{~:]')]

print(tit_bad_fare)

PassengerId
490    15.9a
Name: Fare, dtype: object


Pasażer o id 490 posiada wartość 15.9a. Zostanie ona zmieniona na 15.9.

In [188]:
tit_correct_fare.loc[tit_correct_fare.index == 490, 'Fare'] = '15.9'

print(tit_correct_fare['Fare'][tit_correct_fare.index == 490])

PassengerId
490    15.9
Name: Fare, dtype: object


Aby zweryfikować wartości min i max, należy najpierw zmienić typ danych kolumny *Fare* na float. Po zmianie typu, zaokrąglono także liczby do dwóch miejsc dziesiętnych.

In [189]:
tit_correct_fare['Fare'] = tit_correct_fare['Fare'].astype(float)
tit_correct_fare = tit_correct_fare.round(2)

print(tit_correct_fare['Fare'].unique())

[  7.25  71.28   7.92  53.1    8.05   8.46  51.86  21.08  11.13  30.07
  16.7   26.55   7.85  16.    29.12  13.    18.     7.22  26.     8.03
  31.39 263.     7.88   7.9   27.72 146.52   7.75  10.5   82.17  52.
   7.23  11.24   9.48  21.    41.58  15.5   21.68  17.8   39.69   7.8
  76.73  61.98  35.5   27.75  46.9   80.    83.48  27.9   15.25   8.16
   8.66  73.5   14.45  56.5    7.65  29.    12.48   9.     9.5    7.79
  47.1   15.85  34.38  61.18  20.58  34.65  63.36  23.    77.29   8.65
   7.78  24.15   9.82  14.46 247.52  31.28   7.14  22.36   6.98   7.05
  14.5   15.05  26.28   9.22  79.2    6.75  11.5   36.75  12.52  66.6
   7.31  61.38   7.73  69.55  16.1   15.75  20.52  55.    25.92  33.5
  30.7   25.47  28.71   0.    39.    22.02  50.     8.4    6.5   10.46
  18.79  31.   113.28  27.    76.29  90.     9.35  13.5    7.55  26.25
  12.28   7.12 -90.    52.55  20.21  86.5  512.33  79.65 153.46 135.63
  19.5   29.7   77.96  20.25  78.85  91.08  12.88   8.85 151.55  30.5
  23.25  12.

Następnie wyznaczono wartości min i max.

In [190]:
print("Wartość maksymalna: " + str(tit_correct_fare['Fare'].max()))
print("Wartość minimalna: " + str(tit_correct_fare['Fare'].min()))

Wartość maksymalna: 512.33
Wartość minimalna: -90.0


Cena maksymalna jest wysoka i odstająca od pozostałych, ale poprawna. Natomiast cena minimalna jest błędna. Wyznaczone zostaną wszystkie ceny mniejsze niż 0.

In [191]:
print(tit_correct_fare[tit_correct_fare['Fare']<0]['Fare'])

PassengerId
246   -90.0
Name: Fare, dtype: float64


Cena biletu dla pasażera o ID 246 zostanie zmieniona z -90 na 90.

In [192]:
tit_correct_fare.loc[tit_correct_fare.index == 246, 'Fare'] = 90.0

print(tit_correct_fare['Fare'][tit_correct_fare.index == 246])

PassengerId
246    90.0
Name: Fare, dtype: float64


## Przypisanie poprawionego zbioru do zmiennej wynikowej

Wynik ostatniej poprawy zostanie przypisany do zmiennej *tit_cleansed*, która następnie zostanie zapisana do pliku.

In [193]:
tit_cleansed = tit_correct_fare.copy();

## Zapis wynikowych danych do pliku

Poprawiony zbiór zostanie zapisany do pliku tekstowego z kolumnami oddzielonymi tabulatorem. Plik będzie posiadał nazwę "TitanicCleaned.tsv".

Plik zostanie zapisany w katalogu, w którym znajduje się ten Notebook.

In [194]:
tit_cleansed.to_csv(r'TitanicCleaned.tsv', sep='\t')